In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/reid/deep-person-reid
!python setup.py develop
import torch
from torch import nn
import torchreid
%cd ..

/content/drive/MyDrive/reid/deep-person-reid
running develop
running egg_info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'torchreid.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.7/torchreid/metrics/rank_cylib/rank_cy.cpython-37m-x86_64-linux-gnu.so -> torchreid/metrics/rank_cylib
Creating /usr/local/lib/python3.7/dist-packages/torchreid.egg-link (link to .)
torchreid 1.4.0 is already the active version in easy-install.pth

Installed /content/drive/MyDrive/reid/deep-person-reid
Processing dependencies for torchreid==1.4.0
error: importlib-metadata 4.2.0 is installed but importlib-metadata>=4.4; python_version < "3.10" is required by {'markdown'}
/content/drive/MyDrive/reid


In [3]:
datamanager = torchreid.data.datamanager.ImageDataManager(
    root='',
    sources='dukemtmcreid',
    height=256,
    width=128,
    transforms='random_flip',
    batch_size_train=64,
    batch_size_test=64
)
# return train loader of source data
train_loader = datamanager.train_loader

# return test loader of target data
test_loader = datamanager.test_loader

# return train loader of target data
train_loader_t = datamanager.train_loader_t


Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------


  **************** Summary ****************
  source            : ['dukemtmcreid']
  # source datasets : 1
  # source ids      : 702
  # source images   : 16522
  # source cameras  : 8
  target            : ['dukemtmcreid']
  *****************************************




In [ ]:
for target in list(test_loader.keys()):

  print(test_loader[target]['gallery'])
print(test_loader.values)

<built-in method values of dict object at 0x7f738c9e5190>


Model osnet_x1_0

In [ ]:
model = torchreid.models.osnet_x1_0(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model = model.cuda()

weight_path = 'pretrained/osnet_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model, weight_path)
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0015
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"


In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x1_0_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 0.911 (1.075)	data 0.001 (0.143)	eta 0:46:03	loss 6.1969 (6.3562)	acc 9.3750 (9.6875)	lr 0.001500
epoch: [1/10][20/258]	time 0.908 (0.991)	data 0.001 (0.072)	eta 0:42:17	loss 6.1192 (6.2449)	acc 12.5000 (10.3125)	lr 0.001500
epoch: [1/10][30/258]	time 0.911 (0.964)	data 0.001 (0.048)	eta 0:40:57	loss 5.6831 (6.1175)	acc 10.9375 (10.7292)	lr 0.001500
epoch: [1/10][40/258]	time 0.908 (0.951)	data 0.001 (0.036)	eta 0:40:15	loss 5.0742 (5.9098)	acc 23.4375 (12.7734)	lr 0.001500
epoch: [1/10][50/258]	time 0.916 (0.944)	data 0.001 (0.029)	eta 0:39:48	loss 4.9128 (5.6846)	acc 15.6250 (15.3750)	lr 0.001500
epoch: [1/10][60/258]	time 0.915 (0.939)	data 0.000 (0.024)	eta 0:39:27	loss 4.4704 (5.4864)	acc 28.1250 (17.1875)	lr 0.001500
epoch: [1/10][70/258]	time 0.918 (0.936)	data 0.001 (0.021)	eta 0:39:10	loss 4.3592 (5.3007)	acc 29.6875 (19.5312)	lr 0.001500
epoch: [1/10][80/258]	time 0.916 (0.934)	data 0.001 (0.019)	eta 0:38:55	loss 4.5072 (5.1387)	acc 31.2500 (22.0117

Model osnet_x0_75

In [ ]:
model_1 = torchreid.models.osnet_x0_75(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model_1 = model.cuda()

weight_path_1 = 'pretrained/osnet_x0_75_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model_1, weight_path_1)
optimizer_1 = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0015
)
scheduler_1 = torchreid.optim.build_lr_scheduler(
    optimizer_1,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x0_75_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"


In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model_1, optimizer_1, scheduler=scheduler_1
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x0_75_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 6.339 (4.298)	data 5.644 (3.581)	eta 3:04:05	loss 6.5676 (6.4492)	acc 6.2500 (10.0000)	lr 0.001500
epoch: [1/10][20/258]	time 0.697 (3.523)	data 0.000 (2.815)	eta 2:30:19	loss 6.1019 (6.3455)	acc 9.3750 (9.8438)	lr 0.001500
epoch: [1/10][30/258]	time 9.491 (3.627)	data 8.811 (2.925)	eta 2:34:08	loss 5.3636 (6.2208)	acc 25.0000 (10.0000)	lr 0.001500
epoch: [1/10][40/258]	time 0.719 (3.308)	data 0.000 (2.606)	eta 2:20:03	loss 4.7548 (5.9956)	acc 25.0000 (12.1094)	lr 0.001500
epoch: [1/10][50/258]	time 10.371 (3.334)	data 9.571 (2.633)	eta 2:20:34	loss 4.6849 (5.7727)	acc 28.1250 (14.6875)	lr 0.001500
epoch: [1/10][60/258]	time 0.680 (3.168)	data 0.000 (2.469)	eta 2:13:02	loss 4.9021 (5.5754)	acc 14.0625 (16.7448)	lr 0.001500
epoch: [1/10][70/258]	time 3.684 (3.190)	data 2.987 (2.489)	eta 2:13:25	loss 4.0206 (5.3898)	acc 40.6250 (19.0848)	lr 0.001500
epoch: [1/10][80/258]	time 0.691 (3.093)	data 0.000 (2.394)	eta 2:08:52	loss 3.6964 (5.2176)	acc 45.3125 (21.3672

Model osnet_x0_5

In [ ]:
model_2 = torchreid.models.osnet_x0_5(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model_2 = model.cuda()

weight_path_2 = 'pretrained/osnet_x0_5_duke_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model_2, weight_path_2)
optimizer_2 = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0015
)
scheduler_2 = torchreid.optim.build_lr_scheduler(
    optimizer_2,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x0_5_duke_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth"
** The following layers are discarded due to unmatched keys or layer size: ['conv1.conv.weight', 'conv1.bn.weight', 'conv1.bn.bias', 'conv1.bn.running_mean', 'conv1.bn.running_var', 'conv2.0.conv1.conv.weight', 'conv2.0.conv1.bn.weight', 'conv2.0.conv1.bn.bias', 'conv2.0.conv1.bn.running_mean', 'conv2.0.conv1.bn.running_var', 'conv2.0.conv2a.conv1.weight', 'conv2.0.conv2a.conv2.weight', 'conv2.0.conv2a.bn.weight', 'conv2.0.conv2a.bn.bias', 'conv2.0.conv2a.bn.running_mean', 'conv2.0.conv2a.bn.running_var', 'conv2.0.conv2b.0.conv1.weight', 'conv2.0.conv2b.0.conv2.weight', 'conv2.0.conv2b.0.bn.weight', 'conv2.0.conv2b.0.bn.bias', 'conv2.0.conv2b.0.bn.running_mean', 'conv2.0.conv2b.0.bn.running_var', 'conv2.0.conv2b.1.conv1.weight', 'conv2.0.conv2b.1.conv2.weight', 'conv2.0.conv2b.1.bn.weight', 'conv2.0.conv2b.1.bn.bias', 'conv2.0.conv2b.1.bn.r

In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model_2, optimizer_2, scheduler=scheduler_2
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x0_5_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 0.680 (0.846)	data 0.001 (0.137)	eta 0:36:13	loss 6.1620 (6.2559)	acc 9.3750 (9.2188)	lr 0.001500
epoch: [1/10][20/258]	time 0.685 (0.766)	data 0.001 (0.069)	eta 0:32:41	loss 5.8335 (6.0010)	acc 20.3125 (14.3750)	lr 0.001500
epoch: [1/10][30/258]	time 0.689 (0.739)	data 0.001 (0.046)	eta 0:31:24	loss 5.1065 (5.6916)	acc 26.5625 (18.9062)	lr 0.001500
epoch: [1/10][40/258]	time 0.680 (0.725)	data 0.000 (0.035)	eta 0:30:40	loss 3.9949 (5.3697)	acc 42.1875 (23.0469)	lr 0.001500
epoch: [1/10][50/258]	time 0.682 (0.717)	data 0.001 (0.028)	eta 0:30:13	loss 4.0198 (5.0909)	acc 43.7500 (27.2812)	lr 0.001500
epoch: [1/10][60/258]	time 0.687 (0.711)	data 0.001 (0.023)	eta 0:29:52	loss 3.4674 (4.8489)	acc 50.0000 (31.0417)	lr 0.001500
epoch: [1/10][70/258]	time 0.685 (0.708)	data 0.001 (0.020)	eta 0:29:36	loss 3.0240 (4.5897)	acc 65.6250 (35.6473)	lr 0.001500
epoch: [1/10][80/258]	time 0.695 (0.705)	data 0.000 (0.018)	eta 0:29:23	loss 2.5572 (4.3621)	acc 76.5625 (39.5117

Model osnet_x0_25

In [ ]:
model_3 = torchreid.models.osnet_x0_25(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model_3 = model.cuda()

weight_path_3 = 'pretrained/osnet_x0_25_duke_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model_3, weight_path_3)
optimizer_3 = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0015
)
scheduler_3 = torchreid.optim.build_lr_scheduler(
    optimizer_2,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x0_25_duke_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth"
** The following layers are discarded due to unmatched keys or layer size: ['conv1.conv.weight', 'conv1.bn.weight', 'conv1.bn.bias', 'conv1.bn.running_mean', 'conv1.bn.running_var', 'conv2.0.conv1.conv.weight', 'conv2.0.conv1.bn.weight', 'conv2.0.conv1.bn.bias', 'conv2.0.conv1.bn.running_mean', 'conv2.0.conv1.bn.running_var', 'conv2.0.conv2a.conv1.weight', 'conv2.0.conv2a.conv2.weight', 'conv2.0.conv2a.bn.weight', 'conv2.0.conv2a.bn.bias', 'conv2.0.conv2a.bn.running_mean', 'conv2.0.conv2a.bn.running_var', 'conv2.0.conv2b.0.conv1.weight', 'conv2.0.conv2b.0.conv2.weight', 'conv2.0.conv2b.0.bn.weight', 'conv2.0.conv2b.0.bn.bias', 'conv2.0.conv2b.0.bn.running_mean', 'conv2.0.conv2b.0.bn.running_var', 'conv2.0.conv2b.1.conv1.weight', 'conv2.0.conv2b.1.conv2.weight', 'conv2.0.conv2b.1.bn.weight', 'conv2.0.conv2b.1.bn.bias', 'conv2.0.conv2b.1.bn.

In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model_3, optimizer_3, scheduler=scheduler_3
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x0_25_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 0.678 (0.838)	data 0.001 (0.103)	eta 0:35:53	loss 5.7307 (6.3796)	acc 20.3125 (8.5938)	lr 0.001500
epoch: [1/10][20/258]	time 0.682 (0.760)	data 0.001 (0.052)	eta 0:32:24	loss 4.9320 (5.9818)	acc 26.5625 (14.0625)	lr 0.001500
epoch: [1/10][30/258]	time 0.683 (0.735)	data 0.001 (0.035)	eta 0:31:13	loss 4.7479 (5.6437)	acc 39.0625 (19.8438)	lr 0.001500
epoch: [1/10][40/258]	time 0.691 (0.722)	data 0.001 (0.026)	eta 0:30:33	loss 3.5633 (5.2581)	acc 43.7500 (26.0156)	lr 0.001500
epoch: [1/10][50/258]	time 0.682 (0.714)	data 0.001 (0.021)	eta 0:30:06	loss 3.5305 (4.9358)	acc 53.1250 (30.8750)	lr 0.001500
epoch: [1/10][60/258]	time 0.684 (0.709)	data 0.000 (0.018)	eta 0:29:47	loss 2.8380 (4.6225)	acc 67.1875 (36.3021)	lr 0.001500
epoch: [1/10][70/258]	time 0.678 (0.705)	data 0.001 (0.015)	eta 0:29:29	loss 2.7339 (4.3444)	acc 68.7500 (40.8259)	lr 0.001500
epoch: [1/10][80/258]	time 0.694 (0.702)	data 0.001 (0.014)	eta 0:29:16	loss 2.5211 (4.1108)	acc 70.3125 (44.824

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


epoch: [2/10][10/258]	time 0.685 (0.828)	data 0.001 (0.119)	eta 0:31:54	loss 1.2243 (1.2524)	acc 98.4375 (99.2188)	lr 0.001500
epoch: [2/10][20/258]	time 0.683 (0.757)	data 0.001 (0.060)	eta 0:29:02	loss 1.2687 (1.2692)	acc 98.4375 (98.5156)	lr 0.001500
epoch: [2/10][30/258]	time 0.681 (0.733)	data 0.001 (0.040)	eta 0:27:59	loss 1.2412 (1.2600)	acc 100.0000 (98.8542)	lr 0.001500
epoch: [2/10][40/258]	time 0.682 (0.721)	data 0.001 (0.030)	eta 0:27:24	loss 1.2049 (1.2575)	acc 98.4375 (98.7109)	lr 0.001500
epoch: [2/10][50/258]	time 0.683 (0.714)	data 0.001 (0.024)	eta 0:27:01	loss 1.2181 (1.2517)	acc 96.8750 (98.7188)	lr 0.001500
epoch: [2/10][60/258]	time 0.681 (0.709)	data 0.001 (0.020)	eta 0:26:43	loss 1.2686 (1.2505)	acc 96.8750 (98.6979)	lr 0.001500
epoch: [2/10][70/258]	time 0.683 (0.706)	data 0.001 (0.018)	eta 0:26:28	loss 1.2010 (1.2508)	acc 100.0000 (98.6384)	lr 0.001500
epoch: [2/10][80/258]	time 0.686 (0.703)	data 0.001 (0.016)	eta 0:26:15	loss 1.2232 (1.2531)	acc 100.0000 (98

Evaluation


In [4]:

model_1 = torchreid.models.osnet_x1_0(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model_1 = model_1.cuda()

weight_path = 'log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10'
torchreid.utils.torchtools.load_pretrained_weights(model_1, weight_path)
optimizer_1 = torchreid.optim.build_optimizer(
    model_1, optim='adam', lr=0.0015
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer_1,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10"


In [5]:
engine_1 = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model_1, optimizer_1, scheduler=scheduler
)
engine_1.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=1,
    test_only=True,
    visrank=True,
    visrank_topk=5,
    save_dir='log/osnet_x1_0_dukemtmcreid',
    print_freq=10
)

##### Evaluating dukemtmcreid (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 2228-by-512 matrix
Extracting features from gallery set ...
Done, obtained 17661-by-512 matrix
Speed: 0.0349 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 63.9%
CMC curve
Rank-1  : 83.4%
Rank-5  : 91.7%
Rank-10 : 93.2%
Rank-20 : 95.0%
# query: 2228
# gallery 17661
Visualizing top-5 ranks ...
- done 100/2228
- done 200/2228
- done 300/2228
- done 400/2228
- done 500/2228
- done 600/2228
- done 700/2228
- done 800/2228
- done 900/2228
- done 1000/2228
- done 1100/2228
- done 1200/2228
- done 1300/2228
- done 1400/2228
- done 1500/2228
- done 1600/2228
- done 1700/2228
- done 1800/2228
- done 1900/2228
- done 2000/2228
- done 2100/2228
- done 2200/2228
Done. Images have been saved to "log/osnet_x1_0_dukemtmcreid/visrank_dukemtmcreid" ...


In [ ]:
!python deep-person-reid/tools/visualize_actmap.py --root ./ -d dukemtmcreid -m osnet_x1_0 --weights log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10 --save-dir log/visactmap_osnet_x1_0_dukemtmcreid_test

Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader 